# Title
[]()

In [5]:
import pandas as pd
import sys
import os
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from file_functions import *
import time
import re
from itertools import product
import openai

from response_processing import *
from article_processing import create_text_dict_from_folder
import traceback
from file_functions import *
from summary_chain import *

In [3]:
# set the option to wrap text within cells
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Set up

In [3]:
chatbot_dict = dict()
simple_summaries_dict = dict()
relevance_dict = dict()
chain_results_dict = dict()
qna_dict = dict()
texts = dict()

In [14]:
# Create text dictionary
folder_path = '../web_articles/' # ** UPDATE REQUIRED**
text_id = 0

encoding='ISO-8859-1'
subset=None

texts[text_id] = create_text_dict_from_folder(folder_path, encoding=encoding, subset=subset)
text_dict = texts[text_id]

main_text = dict()
main_text[text_id] = loadpickle('text_dict_from_web2023-06-15_1057.sav', folder_path)


Keys for text_dict: dict_keys([])

Time completed: 2023-06-15 17:22:06.032058
Dictionary keys: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


## text_id = 1 : text_dict with first 2 of 9 articles

In [15]:
text_id = 1
main_text[text_id] = dict()
for article in main_text[0]:
    if article < 2:
        main_text[text_id][article] = main_text[0][article]
main_text[text_id].keys()

dict_keys([0, 1])

In [16]:
main_text[text_id][1]

'Trend of risk and correlates of under-five child undernutrition in Bangladesh: an analysis based on Bangladesh Demographic and Health Survey data, 2007–2017/2018\n\n<div class="panel-panel panel-col">\n    <div><div class="panel-pane pane-highwire-markup author-affiliates col-narrow-12 author-affiliates-corresp article">\n  \n      \n  \n  <div class="pane-content">\n    <div class="highwire-markup"><div xmlns="http://www.w3.org/1999/xhtml" class="content-block-markup" xmlns:xhtml="http://www.w3.org/1999/xhtml"><div xmlns:xhtml="http://www.w3.org/1999/xhtml" class="contributors"><ol class="contributor-list" id="contrib-group-1"><li class="contributor" id="contrib-1"><a href="http://orcid.org/0000-0002-6002-2100" class="bmjj-markup-orcid-logo" target="_blank">http://orcid.org/0000-0002-6002-2100</a><span class="name">Md Tahidur Rahman</span><a id="xref-aff-1-1" class="xref-aff" href="#aff-1">1</a><span class="xref-sep">,</span><a id="xref-aff-2-1" class="xref-aff" href="#aff-2">2</a>, 

# Iteration 1

## prompts

In [7]:
prep_step = [
    "Think about why this might be relevant for the audience in the grand scheme of things.\
    \nIdentify 1 or 2 key concepts from this article that would make interesting or helpful health content. \
    Exclude details that do not add value to the audience.\
    \nBased on the key concepts from the previous steps, extract the key points and result statistics to",
]

summarize_task = [
    "summarize for a LinkedIn post.",
    # "Describe the interesting points to your coworker at the water cooler",
    # "Create an Instagram post without hashtags.",
]
edit_task = [
    "\nIf applicable, include a brief description of the research participants, \
    such as age and sex, within the body of the summary. \
    Otherwise, you can skip this step.\
    \nEvaluate whether or not your writing may be confusing or redundant. \
    \nIf so, re-write it so it is clear and concise. Otherwise, keep it the same. \
    \nCreate an intriguing headline to hook the audience.\
    \nReturn your response in this format:\
    \n<headline>\n\n<summary>\
    \nwhere the summary is a publication-ready format.\
    \nDo not label the headline and summary.",
]

system_role = "You are a journalist writing content based on science research articles."
prompts_df = pd.DataFrame(product(prep_step, summarize_task, edit_task), 
    columns=['prep_step', 'summarize_task', 'edit_task'])

user_simplify_task = [
    """If needed, rewrite the headline and text using terms appropriate for the audience. If not keep it the same.\
    Follow these steps to accomplish this: \
    \n1. Check if the content and language are appropriate for the audience. \
    \n2. If it is suitable for the audience, keep it the same. If not, rewrite using terms appropriate for the audience. \ 
    \n3. Return the final version of the headline and text to be shown to the audience. \
    \nIt should be in this format:\
    \n<headline>\n\n<summary>\
    \n\nwhere the summary is in paragraph form.\
    \n4. Remove the backticks. Do not label the headline and summary. ",
    \n\nYour audience is""",
]

simplify_audience = [
    # "a lay audience",
    "people who are not science experts",
]

user_relevance_task = [
    """Rewrite the headline and text to include a statement of how it is relevant for the audience. \
    Follow these steps to accomplish this: \
    \n1. Think about why this might be relevant for the audience in the grand scheme of things.\
    \n2. If it is not evident why the text is relevant for the audience in the grand scheme of things, \
    add a sentence to inform the audience. Otherwise, keep it the same. \
    Avoid making claims that are not supported by science.\
    \n3. Modify the content if needed to reduce redundancy. \
    \n4. Check if the content and language are appropriate for the audience. \
    If it is suitable for the audience, keep it the same. If not, rewrite using terms appropriate for the audience. \ 
    \n5. Return the final version of the headline and text to be shown to the audience. \
    \nIt should be in this format:\
    \n<headline>\n\n<summary>\
    \n\nwhere the summary is in paragraph form.\
    \n6. Remove the backticks. Do not label the headline and summary. ",
    \n\nYour audience consists of""",
]

relevance_audience = [
    "seniors",
    # "people who enjoy sports",
    # "people new to resistance training",
    "people starting an exercise program"
]

## run

In [23]:
# Set parameters
iteration_id = 1
n_choices = 2
pause_per_request=0
chatbot_id = iteration_id
# chatbot_id = 0
save_outputs = False
save = True
# save = False
empty_columns = True
model = 'gpt-3.5-turbo-16k-0613'
text_dict = main_text[text_id]

# Create initial summaries
chaining_dict = batch_summarize_chain(
    text_dict, folder_path, prep_step, summarize_task, edit_task, chatbot_dict,
    system_role=system_role, model=model, max_tokens=1000,
    n_choices=n_choices, pause_per_request=pause_per_request,
    iteration_id=iteration_id, save_outputs=save_outputs
    )
qna_dict = spreadsheet_columns(
    qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id, save=save
    )

# # Create simple summaries
# audience = simplify_audience
# simple_summaries = prompt_chaining_dict(user_simplify_task, simplify_audience, simple_summaries_dict, 
#     chatbot_dict[chatbot_id], iteration_id,
#     n_choices=1, pause_per_request=pause_per_request, chatbot_id=chatbot_id
#     )

# # Add relevance
# relevance = prompt_chaining_dict(user_relevance_task, relevance_audience, relevance_dict, 
#     chatbot_dict[chatbot_id], iteration_id, prompt_column='relevance', 
#     n_choices=1, pause_per_request=pause_per_request, chatbot_id=chatbot_id
#     )

# # Merge the results
# try:
#     qna_dict = merge_all_chaining_results(
#         chatbot_dict, qna_dict, iteration_id=iteration_id, relevance_audiences=2, pivot=True,
#         empty_columns=empty_columns, chatbot_id=chatbot_id,
#         save_df=save, save_chatbot=save, 
#             csv_path=folder_path,
#     )
#     print(f'\nCompleted merge_all_chaining_results!:)')
# except Exception as error:
#     exc_type, exc_obj, tb = sys.exc_info()
#     f = tb.tb_frame
#     lineno = tb.tb_lineno
#     file = f.f_code.co_filename
#     print(f'An error occurred on line {lineno} in {file}: {error}')
#     print('Unable to merge results')
#     if save:
#         save_instance_to_dict(chatbot_dict[chatbot_id], ext=None, json_path=folder_path)
#         print(f'\nCould not merge; saved Chaining instances as JSON.')

# qna_dict = merge_all_chaining_results2(
#     chatbot_dict, qna_dict, iteration_id=iteration_id, relevance_audiences=2, pivot=True,
#     empty_columns=empty_columns, chatbot_id=chatbot_id,
#         csv_path=folder_path,
# )
# print(f'\nCompleted merge_all_chaining_results!:)')

qna_dict[iteration_id]

**Text #0 prompt #1 of 1**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-16k-0613
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-16k-0613
		Requesting 2 choices using gpt-3.5-turbo-16k-0613
An error occurred on line 145 in C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src\summary_chain.py : This model's maximum context length is 16385 tokens. However, your messages resulted in 55181 tokens. Please reduce the length of the messages.
	**API request failed for `.summarize()`**
	...Completed
**Text #1 prompt #1 of 1**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-16k-0613
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-16k-0613
		Requesting 2 choices using gpt-3.5-turbo-16k-0613
An error occurred on line 145 in C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src\summary_chain.py : This model's maximum context length is 16385 to

KeyError: 'summary'

# `trim_text` function : Need to reduce tokens in messages

In [18]:
text_dict = main_text[text_id]
test_text = text_dict[0]
test_text

'Endovascular thrombectomy with or without thrombolysis bridging in patients with acute ischaemic stroke: protocol for a systematic review, meta-analysis of randomised trials and cost-effectiveness analysis\n\n<div class="panel-panel panel-col">\n    <div><div class="panel-pane pane-highwire-markup author-affiliates col-narrow-12 author-affiliates-corresp article">\n  \n      \n  \n  <div class="pane-content">\n    <div class="highwire-markup"><div xmlns="http://www.w3.org/1999/xhtml" class="content-block-markup" xmlns:xhtml="http://www.w3.org/1999/xhtml"><div xmlns:xhtml="http://www.w3.org/1999/xhtml" class="contributors"><ol class="contributor-list" id="contrib-group-1"><li class="contributor" id="contrib-1"><a href="http://orcid.org/0000-0003-2131-3711" class="bmjj-markup-orcid-logo" target="_blank">http://orcid.org/0000-0003-2131-3711</a><span class="name">Rami Z Morsi</span><a id="xref-aff-1-1" class="xref-aff" href="#aff-1">1</a>, </li><li class="contributor" id="contrib-2"><span

In [26]:
import re
from IPython import display
def trim_text(text, regex=r'(.*)'):
    processed = re.findall(regex, text, re.DOTALL)
    try:
        return display.HTML(processed[0])
    except:
        print('No HTML to display')
        return processed

regex = r'(.*)'
trim_text(test_text, regex)

# Update crawler script

## Iteration 2

In [1]:
import scrapy
from scrapy.crawler import CrawlerRunner
from crochet import setup, wait_for
setup()

from pprint import pprint

class myspider(scrapy.Spider):
    name = "myspider"
    
    def start_requests(self):
        urls = ['https://emails.bmj.com/q/1fnLH65XUsNn7Iiph6kELOM/wv']
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse_front)
    
    def parse_front(self, response):
        self.article_title = response.css('a.art-title > font::text').extract()
        article_url = response.xpath('//a[@class="art-title"]/@href').extract()
        # self.article_title = [response.css('a.art-title > font::text').extract_first()]
        # article_url = [response.xpath('//a[@class="art-title"]/@href').extract_first()]
        for index, url in enumerate(article_url):
            article_dict[index] = dict()
            article_dict[index]['title'] = self.article_title[index]
            article_dict[index]['url'] = url
            yield response.follow(url=url, callback=self.parse_pages, cb_kwargs={'index': index})
    
    def parse_pages(self, response, index):
        text = response.xpath('//h2|//p').extract()
        article_dict[index]['text'] = ''.join([line for line in text])
        
@wait_for(10)
def run_spider():
    crawler = CrawlerRunner()
    d = crawler.crawl(myspider)
    return d


iteration_id = 2
main_dict = dict()
article_dict = dict()

run_spider()
main_dict[iteration_id] = article_dict

pprint(next(iter(article_dict.values()))['text'])

('<p class="logo-bmj-journals"><a href="http://journals.bmj.com/" title="BMJ '
 'Journals"><img '
 'src="/sites/default/themes/bmjj/img/logos/logo-bmj-journals.svg" title="BMJ '
 'Journals" alt="BMJ Journals"></a></p><h2 class="pane-title"><span '
 'class="icon-search"></span></h2><h2 class="pane-title"><span '
 'class="icon-menu"></span></h2><h2 class="pane-title">Main menu</h2><h2 '
 'class="element-invisible">You are here</h2><p><span '
 'class="icon-article-text"></span>Article Text</p><p><a '
 'class="article-pdf-download" href="/content/bmjopen/13/6/e064322.full.pdf" '
 'target="new"><img alt="Download PDF" '
 'src="/sites/default/themes/bmjj/img/icon-pdf.png"><strong>PDF</strong></a></p><p><a '
 'class="article-pdf-download" '
 'href="/content/bmjopen/13/6/e064322.full.pdf?with-ds=yes" target="new"><img '
 'alt="Download PDF + Supplemental Data" '
 'src="/sites/default/themes/bmjj/img/icon-pdf.png"><strong>PDF +<br>\n'
 'Supplementary<br>\n'
 'Material</strong></a></p><h2>Abstra

In [21]:
test_text = next(iter(article_dict.values()))['text']
trim_text(test_text, regex)

['<p class="logo-bmj-journals"><a href="http://journals.bmj.com/" title="BMJ Journals"><img src="/sites/default/themes/bmjj/img/logos/logo-bmj-journals.svg" title="BMJ Journals" alt="BMJ Journals"></a></p><h2 class="pane-title"><span class="icon-search"></span></h2><h2 class="pane-title"><span class="icon-menu"></span></h2><h2 class="pane-title">Main menu</h2><h2 class="element-invisible">You are here</h2><p><span class="icon-article-text"></span>Article Text</p><p><a class="article-pdf-download" href="/content/bmjopen/13/6/e064322.full.pdf" target="new"><img alt="Download PDF" src="/sites/default/themes/bmjj/img/icon-pdf.png"><strong>PDF</strong></a></p><p><a class="article-pdf-download" href="/content/bmjopen/13/6/e064322.full.pdf?with-ds=yes" target="new"><img alt="Download PDF + Supplemental Data" src="/sites/default/themes/bmjj/img/icon-pdf.png"><strong>PDF +<br>\nSupplementary<br>\nMaterial</strong></a></p><h2>Abstract</h2><p id="p-2"><strong>Introduction</strong> Current publish

In [22]:
display.HTML(test_text)

In [25]:
main_dict[iteration_id][0]['url']

'https://emails.bmj.com/c/1aDifdDrdRVuIA6ezO2vHCh8xqY4'

## 2.1

In [31]:
import scrapy
from scrapy.crawler import CrawlerRunner
from crochet import setup, wait_for
setup()

from pprint import pprint

class myspider(scrapy.Spider):
    name = "myspider"
    
    def start_requests(self):
        urls = ['https://emails.bmj.com/q/1fnLH65XUsNn7Iiph6kELOM/wv']
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse_front)
    
    def parse_front(self, response):
        # self.article_title = response.css('a.art-title > font::text').extract()
        # article_url = response.xpath('//a[@class="art-title"]/@href').extract()
        self.article_title = [response.css('a.art-title > font::text').extract_first()]
        article_url = [response.xpath('//a[@class="art-title"]/@href').extract_first()]
        for index, url in enumerate(article_url):
            article_dict[index] = dict()
            article_dict[index]['title'] = self.article_title[index]
            article_dict[index]['url'] = url
            yield response.follow(url=url, callback=self.parse_pages, cb_kwargs={'index': index})
    
    def parse_pages(self, response, index):
        text = response.xpath('//h2|//p|//h3').extract()
        article_dict[index]['text'] = ''.join([line for line in text])
        
@wait_for(10)
def run_spider():
    crawler = CrawlerRunner()
    d = crawler.crawl(myspider)
    return d


iteration_id = 2.1
# main_dict = dict()
article_dict = dict()

run_spider()
main_dict[iteration_id] = article_dict

pprint(next(iter(article_dict.values()))['text'])

('<p class="logo-bmj-journals"><a href="http://journals.bmj.com/" title="BMJ '
 'Journals"><img '
 'src="/sites/default/themes/bmjj/img/logos/logo-bmj-journals.svg" title="BMJ '
 'Journals" alt="BMJ Journals"></a></p><h3>Log in using your username and '
 'password</h3><h2 class="pane-title"><span '
 'class="icon-search"></span></h2><h2 class="pane-title"><span '
 'class="icon-menu"></span></h2><h2 class="pane-title">Main menu</h2><h3>Log '
 'in using your username and password</h3><h2 class="element-invisible">You '
 'are here</h2><p><span class="icon-article-text"></span>Article Text</p><p><a '
 'class="article-pdf-download" href="/content/bmjopen/13/6/e064322.full.pdf" '
 'target="new"><img alt="Download PDF" '
 'src="/sites/default/themes/bmjj/img/icon-pdf.png"><strong>PDF</strong></a></p><p><a '
 'class="article-pdf-download" '
 'href="/content/bmjopen/13/6/e064322.full.pdf?with-ds=yes" target="new"><img '
 'alt="Download PDF + Supplemental Data" '
 'src="/sites/default/themes/bmjj

## **2.2 This version of crawler

In [33]:
import scrapy
from scrapy.crawler import CrawlerRunner
from crochet import setup, wait_for
setup()

from pprint import pprint

class myspider(scrapy.Spider):
    name = "myspider"
    
    def start_requests(self):
        urls = ['https://emails.bmj.com/q/1fnLH65XUsNn7Iiph6kELOM/wv']
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse_front)
    
    def parse_front(self, response):
        # self.article_title = response.css('a.art-title > font::text').extract()
        # article_url = response.xpath('//a[@class="art-title"]/@href').extract()
        self.article_title = [response.css('a.art-title > font::text').extract_first()]
        article_url = [response.xpath('//a[@class="art-title"]/@href').extract_first()]
        for index, url in enumerate(article_url):
            article_dict[index] = dict()
            article_dict[index]['title'] = self.article_title[index]
            article_dict[index]['url'] = url
            yield response.follow(url=url, callback=self.parse_pages, cb_kwargs={'index': index})
    
    def parse_pages(self, response, index):
        text = response.xpath('//h2|//p|//h3|//h4').extract()
        article_dict[index]['text'] = ''.join([line for line in text])
        
@wait_for(10)
def run_spider():
    crawler = CrawlerRunner()
    d = crawler.crawl(myspider)
    return d


iteration_id = 2.1
# main_dict = dict()
article_dict = dict()

run_spider()
main_dict[iteration_id] = article_dict

pprint(next(iter(article_dict.values()))['text'])

('<p class="logo-bmj-journals"><a href="http://journals.bmj.com/" title="BMJ '
 'Journals"><img '
 'src="/sites/default/themes/bmjj/img/logos/logo-bmj-journals.svg" title="BMJ '
 'Journals" alt="BMJ Journals"></a></p><h3>Log in using your username and '
 'password</h3><h2 class="pane-title"><span '
 'class="icon-search"></span></h2><h2 class="pane-title"><span '
 'class="icon-menu"></span></h2><h2 class="pane-title">Main menu</h2><h3>Log '
 'in using your username and password</h3><h2 class="element-invisible">You '
 'are here</h2><p><span class="icon-article-text"></span>Article Text</p><p><a '
 'class="article-pdf-download" href="/content/bmjopen/13/6/e064322.full.pdf" '
 'target="new"><img alt="Download PDF" '
 'src="/sites/default/themes/bmjj/img/icon-pdf.png"><strong>PDF</strong></a></p><p><a '
 'class="article-pdf-download" '
 'href="/content/bmjopen/13/6/e064322.full.pdf?with-ds=yes" target="new"><img '
 'alt="Download PDF + Supplemental Data" '
 'src="/sites/default/themes/bmjj

In [35]:
regex = r'.*(<h2>Abstract</h2>.*References<).*'
trim_text(next(iter(article_dict.values()))['text'], regex=regex)

# Update `text_dict_from_web` function to trim text

In [39]:
article_dict.keys()

dict_keys([0])

In [45]:
texts.keys()

dict_keys([0])

## Regex 1

In [49]:
import re
from IPython import display
def trim_text(text, regex=r'(.*)'):
    processed = re.findall(regex, text, re.DOTALL)
    return processed, display.HTML(processed[0])
    
def text_dict_from_web(article_dict, regex=r'(.*)', to_display=0):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    text_dict = dict()
    display_dict = dict()
    to_display = [to_display] if type(to_display) != list else display
    for article_key in article_dict:
        # print(f'Article key: {article_key}')
        trimmed_text, display = trim_text(article_dict[article_key]['text'], regex)
        text_dict[article_key] = f"{article_dict[article_key]['title']}\n\n{trimmed_text}"
        if article_key in to_display:
            display_dict[article_key] = display
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display
    
regex = r'.*(<h2>Abstract</h2>.*References<).*'
text_id = 2
texts[text_id], display = text_dict_from_web(article_dict, regex=regex, to_display=0)
display

text_dict keys: [0]


## Regex 2 Trim away the Abstract

In [52]:
import re
from IPython import display
def trim_text(text, regex=r'(.*)'):
    processed = re.findall(regex, text, re.DOTALL)
    return processed, display.HTML(processed[0])
    
def text_dict_from_web(article_dict, regex=r'(.*)', to_display=0):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    text_dict = dict()
    display_dict = dict()
    to_display = [to_display] if type(to_display) != list else display
    for article_key in article_dict:
        # print(f'Article key: {article_key}')
        trimmed_text, display = trim_text(article_dict[article_key]['text'], regex)
        text_dict[article_key] = f"{article_dict[article_key]['title']}\n\n{trimmed_text}"
        if article_key in to_display:
            display_dict[article_key] = display
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display
    
regex = r'.*<h2>Abstract</h2>.*(?:Introduction)?.*<h2.*?>(Introduction</h2>.*References<).*'
text_id = 2
texts[text_id], display = text_dict_from_web(article_dict, regex=regex, to_display=0)
display

text_dict keys: [0]


## Regex 3 use any header

In [56]:
import re
from IPython import display
def trim_text(text, regex=r'(.*)'):
    processed = re.findall(regex, text, re.DOTALL)
    return processed, display.HTML(processed[0])
    
def text_dict_from_web(article_dict, regex=r'(.*)', to_display=0):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    text_dict = dict()
    display_dict = dict()
    to_display = [to_display] if type(to_display) != list else display
    for article_key in article_dict:
        # print(f'Article key: {article_key}')
        trimmed_text, display = trim_text(article_dict[article_key]['text'], regex)
        text_dict[article_key] = f"{article_dict[article_key]['title']}\n\n{trimmed_text}"
        if article_key in to_display:
            display_dict[article_key] = display
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display
    
regex = r'.*<h\d>Abstract</h\d>.*(?:Introduction)?.*(<h\d.*?>Introduction</h\d>.*References)<.*'
text_id = 2
texts[text_id], display = text_dict_from_web(article_dict, regex=regex, to_display=0)
display

text_dict keys: [0]


In [59]:
print(type(texts[text_id][0]))
texts[text_id][0]

<class 'str'>


'Endovascular thrombectomy with or without thrombolysis bridging in patients with acute ischaemic stroke: protocol for a systematic review, meta-analysis of randomised trials and cost-effectiveness analysis\n\n[\'<h2 class="">Introduction</h2><p id="p-10">Intravenous thrombolysis (IVT) has been a long-standing, evidence-based treatment approach for acute ischaemic stroke.<a id="xref-ref-1-1" class="xref-bibr" href="#ref-1">1 2</a> However, IVT must be delivered within 4.5\\u2009hours, has many contraindications, may not provide adequate reperfusion, especially in patients with large vessel occlusions, and may even increase the risk of intracranial haemorrhage.<a id="xref-ref-3-1" class="xref-bibr" href="#ref-3">3</a> As such, trials were conducted to evaluate the use of endovascular therapy in patients with acute ischaemic stroke. The Interventional Management of Stroke (IMS) III was a randomised controlled trial (RCT) comparing endovascular therapy after IVT with IVT alone. Endovascul

# *End of Page*